In [24]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score
from datasets import load_dataset

In [25]:
ds = load_dataset('mlburnham/PolNLI')
train = ds['train'].to_pandas()
test = pd.read_csv('../data/polnli_test_results.csv')

Using the latest cached version of the dataset since mlburnham/PolNLI couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\mikeb\.cache\huggingface\datasets\mlburnham___pol_nli\default\0.0.0\3d563e282efafcfed852d99d41e6a3e578a77550 (last modified on Tue Aug  6 23:43:00 2024).


In [11]:
def metrics(df, preds, group_by=None):
    true_col = 'entailment'
    
    def get_metrics(y_true, y_pred):
        return {
            'MCC': matthews_corrcoef(y_true, y_pred),
            'Accuracy': accuracy_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred, average='weighted')
        }
    
    results = []
    
    if group_by not in ['dataset', 'task']:
        for col in preds:
            metrics = get_metrics(df[true_col], df[col])
            metrics['Column'] = col
            results.append(metrics)
    else:
        for col in preds:
            for group_name, group in df.groupby(group_by):
                metrics = get_metrics(group[true_col], group[col])
                metrics['Column'] = col
                metrics[group_by.capitalize()] = group_name
                results.append(metrics)
    
    results_df = pd.DataFrame(results)
    
    if group_by in ['dataset', 'task']:
        return results_df.set_index(['Column', group_by.capitalize()])
    else:
        return results_df.set_index('Column')

# Base Generic NLI

In [4]:
model = "MoritzLaurer/deberta-v3-base-zeroshot-v2.0"
pipe = pipeline("zero-shot-classification", model = model, device = 0, batch_size = 32)

In [5]:
%%time
colname = 'base_nli'
test[colname] = 0

for i in test.index:
    hypothesis = test.loc[i, 'augmented_hypothesis']
    sample = test.loc[i, 'premise']
    res = pipe(sample, hypothesis, hypothesis_template = '{}', multi_label = False)
    test.loc[i, colname] = round(res['scores'][0])
test[colname].replace({0:1, 1:0}, inplace = True)

test[colname] = test[colname].astype(int)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


CPU times: total: 4min 9s
Wall time: 4min 37s


<timed exec>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




# Large Generic NLI

In [6]:
model = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
pipe = pipeline("zero-shot-classification", model = model, device = 0, batch_size = 32)

In [7]:
%%time
colname = 'large_nli'
test[colname] = 0

for i in test.index:
    hypothesis = test.loc[i, 'augmented_hypothesis']
    sample = test.loc[i, 'premise']
    res = pipe(sample, hypothesis, hypothesis_template = '{}', multi_label = False)
    test.loc[i, colname] = round(res['scores'][0])
test[colname].replace({0:1, 1:0}, inplace = True)

test[colname] = test[colname].astype(int)

CPU times: total: 7min 42s
Wall time: 8min 24s


<timed exec>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




# Base PolNLI

In [8]:
model = "training_base/best_checkpoint_augmented"
pipe = pipeline("zero-shot-classification", model = model, device = 0, batch_size = 32)

In [9]:
%%time
colname = 'base_polnli'
test[colname] = 0

for i in test.index:
    hypothesis = test.loc[i, 'augmented_hypothesis']
    sample = test.loc[i, 'premise']
    res = pipe(sample, hypothesis, hypothesis_template = '{}', multi_label = False)
    test.loc[i, colname] = round(res['scores'][0])
test[colname].replace({0:1, 1:0}, inplace = True)

test[colname] = test[colname].astype(int)

CPU times: total: 4min 5s
Wall time: 4min 28s


<timed exec>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




# Large Pol NLI

In [4]:
model = "training_large/checkpoint-128462"
pipe = pipeline("zero-shot-classification", model = model, device = 0, batch_size = 64)

In [5]:
%%time
colname = 'large_polnli2'
test[colname] = 0

for i in test.index:
    hypothesis = test.loc[i, 'augmented_hypothesis']
    sample = test.loc[i, 'premise']
    res = pipe(sample, hypothesis, hypothesis_template = '{}', multi_label = False)
    test.loc[i, colname] = round(res['scores'][0])
test[colname].replace({0:1, 1:0}, inplace = True)

test[colname] = test[colname].astype(int)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


CPU times: total: 7min 43s
Wall time: 8min 23s


<timed exec>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [6]:
models = ['base_nli', 'large_nli', 'base_polnli', 'large_polnli', 'large_polnli2', 'llama']
metrics(test, preds = models, group_by = None)

,MCC,Accuracy,F1
Column,,,
base_nli,0.658454,0.834635,0.830205
large_nli,0.731021,0.869062,0.866256
base_polnli,0.894269,0.948978,0.948852
large_polnli,0.915911,0.959326,0.959180
large_polnli2,0.907246,0.955226,0.955123
llama,0.730997,0.862358,0.863467


In [7]:
metrics(test, preds = models, group_by = 'task')

MCC  Accuracy        F1
Column        Task                                                 
base_nli      event extraction         0.528918  0.746858  0.745710
              hatespeech and toxicity  0.494319  0.844437  0.822392
              stance detection         0.553824  0.786101  0.781703
              topic classification     0.875762  0.937653  0.937047
large_nli     event extraction         0.718805  0.850209  0.850508
              hatespeech and toxicity  0.571824  0.861426  0.854152
              stance detection         0.612922  0.811736  0.805470
              topic classification     0.899292  0.949634  0.949257
base_polnli   event extraction         0.813742  0.906774  0.907042
              hatespeech and toxicity  0.841410  0.946036  0.945248
              stance detection         0.920600  0.961546  0.961488
              topic classification     0.926241  0.963834  0.963826
large_polnli  event extraction         0.819699  0.909567  0.909838
              hatespeech and toxicity  0.881535  0.959360  0.959026
              stance detection         0.969009  0.984979  0.984972
              topic classification     0.924496  0.962503  0.962322
large_polnli2 event extraction         0.759235  0.877793  0.878247
              hatespeech and toxicity  0.874411  0.955696  0.955981
              stance detection         0.966128  0.983577  0.983576
              topic classification     0.944425  0.972709  0.972664
llama         event extraction         0.808244  0.905726  0.905480
              hatespeech and toxicity  0.559060  0.782145  0.799067
              stance detection         0.605609  0.798117  0.799651
              topic classification     0.918734  0.959396  0.959505

In [86]:
metrics(test, preds = models, group_by = 'dataset').tail(50)

MCC  \
Column        Dataset                                                   
base_polnli   mlburnham/acled_event_entailment               0.867357   
              mlburnham/argument_quality_ranking_entailment  0.960300   
              mlburnham/bill_summary_entailment              0.907231   
              mlburnham/dehumanizing_hatespeech_entailment   0.816036   
              mlburnham/dem_rep_party_platform_topics        0.938602   
              mlburnham/ibm_claimstance_entailment           0.974825   
              mlburnham/ibm_claimstance_topic_entailment     0.923211   
              mlburnham/polistance_issue_tweets              1.000000   
              mlburnham/scad_event_entailment                0.712124   
              mlburnham/targeted_hatespeech_entailment       0.657270   
              mlburnham/violent_hatespeech_entailment        0.862233   
large_polnli  mlburnham/PoliStance_Affect                    0.968691   
              mlburnham/PoliStance_Affect_QT                 0.849912   
              mlburnham/acled_event_entailment               0.898686   
              mlburnham/argument_quality_ranking_entailment  0.984155   
              mlburnham/bill_summary_entailment              0.922941   
              mlburnham/dehumanizing_hatespeech_entailment   0.894813   
              mlburnham/dem_rep_party_platform_topics        0.957413   
              mlburnham/ibm_claimstance_entailment           0.987450   
              mlburnham/ibm_claimstance_topic_entailment     0.983420   
              mlburnham/polistance_issue_tweets              0.697486   
              mlburnham/scad_event_entailment                0.721980   
              mlburnham/targeted_hatespeech_entailment       0.758879   
              mlburnham/violent_hatespeech_entailment        0.878710   
large_polnli2 mlburnham/PoliStance_Affect                    0.967959   
              mlburnham/PoliStance_Affect_QT                 0.881012   
              mlburnham/acled_event_entailment               0.890183   
              mlburnham/argument_quality_ranking_entailment  0.985732   
              mlburnham/bill_summary_entailment              0.877843   
              mlburnham/dehumanizing_hatespeech_entailment   0.853060   
              mlburnham/dem_rep_party_platform_topics        0.954485   
              mlburnham/ibm_claimstance_entailment           0.993716   
              mlburnham/ibm_claimstance_topic_entailment     0.961436   
              mlburnham/polistance_issue_tweets              1.000000   
              mlburnham/scad_event_entailment                0.688974   
              mlburnham/targeted_hatespeech_entailment       0.788456   
              mlburnham/violent_hatespeech_entailment        0.888161   
llama         mlburnham/PoliStance_Affect                    0.529900   
              mlburnham/PoliStance_Affect_QT                 0.097657   
              mlburnham/acled_event_entailment               0.822502   
              mlburnham/argument_quality_ranking_entailment  0.939281   
              mlburnham/bill_summary_entailment              0.891482   
              mlburnham/dehumanizing_hatespeech_entailment   0.753413   
              mlburnham/dem_rep_party_platform_topics        0.932305   
              mlburnham/ibm_claimstance_entailment           0.601317   
              mlburnham/ibm_claimstance_topic_entailment     0.944698   
              mlburnham/polistance_issue_tweets              0.275092   
              mlburnham/scad_event_entailment                0.778034   
              mlburnham/targeted_hatespeech_entailment       0.311477   
              mlburnham/violent_hatespeech_entailment        0.605713   

                                                             Accuracy  \
Column        Dataset                                                   
base_polnli   mlburnham/acled_event_entailment               0.932893   
              mlburnham/argument_quality_ranking_e

In [95]:
test.to_csv('../data/polnli_test_results.csv', index = False)